In [1]:
import os
print(os.getcwd())

/home/mk009/DA6401/ae21b036_da6401_assignment-01/src/ann


In [2]:
import sys
sys.path.append(os.path.abspath(".."))

In [3]:
from neural_network import *

In [4]:
import argparse 

parser = argparse.ArgumentParser()
parser.add_argument('-d', choices=['mnist', 'fashion_mnist'], default= 'mnist', help= '--dataset: \'mnist\' or \'fashion_mnist\'')
parser.add_argument('-e', type= int, default= 1000, help= '--epochs: Number of training epochs')
parser.add_argument('-b', type= int, default= 64, help= '--batch_size: Mini-batch size')
parser.add_argument('-lr', type= float, default= 1e-3, help= '--learning_rate: Learning rate for optimizer')
parser.add_argument('-o', choices=['sgd', 'momentum', 'nag', 'rmsprop', 'adam', 'nadam'], default='sgd', help= '--optimizer: \'sgd\', \'momentum\', \'nag\', \'rmsprop\', \'adam\', \'nadam\'' )
parser.add_argument('-nhl', type=int, default=2, help= '--num_layers: Number of hidden layers')
parser.add_argument('-sz', type=int, nargs='+', help= '--hidden_size: Number of neurons in each hidden layer (list of values)')
parser.add_argument('-a', choices= ['sigmoid', 'tanh', 'relu'], default='sigmoid', help= '--activation: choice of sigmoid, tanh, relu')
parser.add_argument('-l', choices=['mean_squared_error', 'cross_entropy'], default='mean_squared_error', help= 'Loss function (\'cross_entropy\', \'mse\')')
parser.add_argument('-w_i', choices= ['random', 'xavier'], default='random', help= '--weight_init: choice of random or xavier')
parser.add_argument('-wd', type= float, default=0., help= '--weight_decay: Weight decay for L2 regularization')
parser.add_argument('-wb_project', default= '', help= '--wandb_project: W&B project name')
parser.add_argument('-save_path', default='', help= '--model_save_path: Path to save trained model')

_StoreAction(option_strings=['-save_path'], dest='save_path', nargs=None, const=None, default='', type=None, choices=None, required=False, help='--model_save_path: Path to save trained model', metavar=None)

In [5]:
args = parser.parse_args(args=[])

In [6]:
args.nhl = 2
args.sz = [512,512]
args.l = 'cross_entropy'
args.wd = 0.001
args.a = 'relu'

In [7]:
from tensorflow.keras import datasets

2026-02-23 18:26:48.480106: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-23 18:26:48.519946: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-23 18:26:49.626535: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [8]:
(x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()

In [9]:
NN = NeuralNetwork(28*28, 10, Softmax, args)

In [ ]:
NN.optimizer.lr = 0.1
NN.train(x_train,y_train,100,256,True)


Epoch [1/100]
  Batch [211/211] Loss: 0.856767
Epoch [1/100] Train Loss: 1.933117 | Val Loss: 0.844195 | Val Acc: 0.7488

Epoch [2/100]
  Batch [211/211] Loss: 0.403944
Epoch [2/100] Train Loss: 0.565074 | Val Loss: 0.431198 | Val Acc: 0.8802

Epoch [3/100]
  Batch [211/211] Loss: 0.283501
Epoch [3/100] Train Loss: 0.382807 | Val Loss: 0.354294 | Val Acc: 0.8985

Epoch [4/100]
  Batch [211/211] Loss: 0.390203
Epoch [4/100] Train Loss: 0.325683 | Val Loss: 0.318059 | Val Acc: 0.9122

Epoch [5/100]
  Batch [211/211] Loss: 0.289086
Epoch [5/100] Train Loss: 0.287800 | Val Loss: 0.285691 | Val Acc: 0.9192

Epoch [6/100]
  Batch [211/211] Loss: 0.229086
Epoch [6/100] Train Loss: 0.251798 | Val Loss: 0.252715 | Val Acc: 0.9275

Epoch [7/100]
  Batch [211/211] Loss: 0.198144
Epoch [7/100] Train Loss: 0.221233 | Val Loss: 0.239240 | Val Acc: 0.9313

Epoch [8/100]
  Batch [211/211] Loss: 0.134899
Epoch [8/100] Train Loss: 0.195080 | Val Loss: 0.201549 | Val Acc: 0.9427

Epoch [9/100]
  Batch [

KeyboardInterrupt: 

In [13]:
eval_dataloader = Dataloader(x_test,y_test,64,False,True,True)
total_loss = 0.0
total_correct = 0
total_samples = 0
num_batches = 0

print("\nEvaluation")

total_loss = 0
total_correct = 0
total_samples = 0

for i, (X, y) in enumerate(eval_dataloader):

    y_hat = NN.forward(X)

    loss = NN.objective.loss(y, y_hat)

    total_loss += np.sum(loss)

    preds = np.argmax(y_hat, axis=1)
    true_labels = np.argmax(y, axis=1)

    total_correct += np.sum(preds == true_labels)
    total_samples += y.shape[0]

avg_loss = total_loss / total_samples
accuracy = total_correct / total_samples

print(f"\nEval Loss: {avg_loss:.6f} | Accuracy: {accuracy:.4f}")


Evaluation

Eval Loss: 0.066549 | Accuracy: 0.9799


In [ ]:
print("Layer 0 grad max:", np.argmax((NN.Layers[0].grad_W)),axis)
print("Layer 1 grad max:", np.max(np.abs(NN.Layers[1].grad_W)))

print("Layer 0 input max:", np.max(np.abs(NN.Layers[0].hk_1)))

Layer 0 grad max: 194545
Layer 1 grad max: 0.00021254309035347027
Layer 0 input max: 1.0
